# Lectura de datos avro
Primero que todo, como nos interesa los datos de eventos transitorios, nos encontramos con una pagina web (https://ztf.uw.edu/alerts/public/) la cual es un telescopio que mapea cada día el cielo nocturno para obtener imagenes de eventos transitorios. Estos datos vienen en formato avro los cuales son archivos que almacenan datos en formato binario utilizando el esquema de los datos en un archivo contenedor.

### Caracteristicas clave de los archivos avro
- Serialización de datos:
Avro serializa datos en un formato binario compacto y eficiente. 
- Esquemas definidos:
Los datos están asociados con un esquema definido en formato JSON, lo que facilita la lectura y el procesamiento de los datos. 
- Evolución del esquema:
Avro permite la evolución de los esquemas sin problemas, lo que significa que los programas antiguos pueden leer datos nuevos y viceversa. 
- Compatibilidad con diferentes lenguajes:
Avro ofrece API para varias plataformas, incluyendo Java, Python, Ruby, C, C++ y más. 
- Integración con Hadoop:
Avro es ampliamente utilizado en el ecosistema Hadoop para el almacenamiento y procesamiento de datos. 
- Ideal para streaming:
Avro es adecuado para la transmisión de datos entre sistemas, ya que serializa los datos de manera independiente por filas. 
### Uso de archivos Avro:
- Almacenamiento de datos:
Avro es un formato de archivo útil para almacenar datos de forma persistente en sistemas de almacenamiento distribuido. 
- Transferencia de datos:
Avro facilita la transferencia de datos entre sistemas y aplicaciones, especialmente en entornos de streaming. 
- Procesamiento de datos:
Avro puede ser utilizado para el procesamiento de datos en paralelo, aprovechando las capacidades de los frameworks de computación distribuida como Apache Hadoop. 
- Integración con Kafka:
Apache Kafka utiliza Avro para la serialización de mensajes, lo que permite la transmisión eficiente de datos. 

#### Información importante 
En nuestro caso nos benefició el formato avro ya que al compactarlos en formato binario, lo que nos entregan los archivo es un cubo 3d de imagenes 21x21 en el cual también se traen los metadatos, por lo tanto como nos interesa entrenar una red neuronal con capas convolucionales y densas, decidimos  extraer los datos de las imagenes (para la capa conv) y el metadata (para la capa densa). 

In [1]:
import os
import glob
import gzip
from io import BytesIO
import numpy as np
import pandas as pd
from astropy.io import fits
import fastavro

In [ ]:

def store_ztf_stamps_3d_with_metadata(input_dir, output_dir="ztf_3d_stamps", crop_to_21x21=True):
    """ store_ztf_stamps_3d_with_metadata se encarga de leer archivos AVRO de alertas
     astronomicas del ZTF, extrae las imagenes (las de ciencia, referencia y diferencia)
     las convierte en imagenes 2D de 21x21 pixeles, los apila en un arreglo 3D y guarda 
     todo al final como un archivo .npy junto a los metadatos
     
     input:
     input_dir: corresponde al directorio donde se encuentran los archivos.avro
     
     output:
     un archivo llamado alert_{object_id}_full.npy donde el objet_id es el id del evento 
     transitorio (los cuales seran guardados en el directorio ztf_3d_stamps)"""
    
    os.makedirs(output_dir, exist_ok=True)
    # Nos aseguramos de que haya un archivo.avro
    avro_files = glob.glob(os.path.join(input_dir, "*.avro"))
    if not avro_files:
        print(f"No se encontraron archivos Avro en {input_dir}")
        return

    print(f"Procesando {len(avro_files)} archivos Avro en {input_dir}")
    # Recorre los archivos avro de la carpeta
    for i, avro_file in enumerate(avro_files, 1):

        if i % 500 == 0: 
            print(f"Leyendo archivo {i}/{len(avro_files)}: {avro_file}     ", end='\r')
        try:
            with open(avro_file, 'rb') as f:
                reader = fastavro.reader(f)
                for alert in reader:
                    object_id = alert.get('objectId', 'Unknown')
                    candidate = alert.get('candidate', {})

                    science_stamp = alert.get('cutoutScience', {}).get('stampData', None)
                    reference_stamp = alert.get('cutoutTemplate', {}).get('stampData', None)
                    difference_stamp = alert.get('cutoutDifference', {}).get('stampData', None)

                    def decode_stamp(stamp_data): 
                        """ decode_stamp verifica que haya datos con un if, descomprime los datos 
                        los cuales vienen en formato gzip, abre el contenido FITS con astropy
                        convierte los datos de los FITS en un arreglo NumPy si que es que el arreglo no tiene 
                        21x21 pixeles los recorta, reemplaza los nan por cero y finalmente devuelve el valor
                        limpio

                        input: 
                        stamp_data: los datos de las imagenes

                        output: 
                        retorna una imagen 2D de tamaño (63,63) o (21,21).
                        si hay error o el tamaño no es el esperado: None """
                        
                        if stamp_data:
                            try:
                                decompressed = gzip.decompress(stamp_data)
                                with fits.open(BytesIO(decompressed), ignore_missing_simple=True) as hdu:
                                    arr = hdu[0].data.astype(np.float32)
                                if arr.shape != (63, 63):
                                    
                                    return None
                                if crop_to_21x21:
                                    arr = arr[21:42, 21:42]
                                return np.nan_to_num(arr, nan=0.0)
                            except Exception as e:
                                print(f"Error decodificando stamp: {e}")
                        return None

                    sci = decode_stamp(science_stamp)
                    ref = decode_stamp(reference_stamp)
                    diff = decode_stamp(difference_stamp)

                    if sci is not None and ref is not None and diff is not None:
                        stamps_3d = np.stack([sci, ref, diff])

                        # Recolecta metadatos relevantes
                        metadata = {
                            "ra": candidate.get("ra", -999),
                            "dec": candidate.get("dec", -999),
                            "magpsf": candidate.get("magpsf", -999),
                            "sigmapsf": candidate.get("sigmapsf", -999),
                            "isdiffpos": int(candidate.get("isdiffpos", 'f') == 't'),
                            "diffmaglim": candidate.get("diffmaglim", -999),
                            "fwhm": candidate.get("fwhm", -999),
                            "sgscore1": candidate.get("sgscore1", -999),
                            "sgscore2": candidate.get("sgscore2", -999),
                            "sgscore3": candidate.get("sgscore3", -999),
                            "distpsnr1": candidate.get("distpsnr1", -999),
                            "distpsnr2": candidate.get("distpsnr2", -999),
                            "distpsnr3": candidate.get("distpsnr3", -999),
                            "classtar": candidate.get("classtar", -999),
                            "ndethist": candidate.get("ndethist", -1),
                            "ncovhist": candidate.get("ncovhist", -1),
                            "chinr": candidate.get("chinr", -999),
                            "sharpnr": candidate.get("sharpnr", -999),
                            "gal_lat": alert.get("gal_lat", -999),
                            "gal_lng": alert.get("gal_lng", -999),
                            "ecl_lat": alert.get("ecl_lat", -999),
                            "ecl_lng": alert.get("ecl_lng", -999),
                            "approx_nondet": candidate.get("ncovhist", -1) - candidate.get("ndethist", -1),
                        }

                        # Finalmente guarda como dict
                        data_dict = {
                            "object_id": object_id,
                            "data": stamps_3d,
                            "metadata": metadata
                        }

                        np.save(os.path.join(output_dir, f"alert_{object_id}_full.npy"), data_dict)


        except Exception as e:
            print(f"Error en archivo {avro_file}: {e}")

if __name__ == "__main__":
    input_directory = "/home/seba/Usach/bigdata/Proyecto/datos1"  # Reemplaza con la ruta al directorio con archivos Avro
    store_ztf_stamps_3d_with_metadata(input_directory, output_dir="/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps_1", crop_to_21x21=True)

Procesando 2549 archivos Avro en /home/seba/Usach/bigdata/Proyecto/datos1


### ¿Qué contiene cada .npy?
contiene una especie de diccionario en donde se tienen los objet_id que será llamado como "ZTF_algo...", la data que será el array de imagenes (3x21x21), y finalmente el metadato que seran todos los metadatos que se nombraron en la función.

In [2]:
def normalize_data(data):
    """
    Normaliza los datos de entrada a un rango de 0 a 1.
    """
    data_min = np.min(data)
    data_max = np.max(data)
    return (data - data_min) / (data_max)

# Datos etiquetados 
Como nos interesa crear una red neuronal con capas convolucionales y densas para clasificar eventos transitorios necesitamos de datos etiquetados, los cuales pudimos encontrar en  https://zenodo.org/records/4279623 para luego realizar un crossmatch con nuestros datos a partir del id  

In [ ]:
alerce_data = pd.read_csv("/home/seba/Usach/bigdata/Proyecto/etiquetas/ALeRCE_lc_classifier_outputs_ZTF_unlabeled_set_20200609.csv")

In [4]:
# Muestra las etiquetas únicas y su número total en alerce_data
print("\nEtiquetas en alerce_data['predicted_class']:")
print(alerce_data['predicted_class'].value_counts())


Etiquetas en alerce_data['predicted_class']:
predicted_class
Periodic-Other    243374
E                 198122
LPV               161592
YSO                85087
RRL                58592
QSO                43054
DSCT               26672
CEP                17307
AGN                14342
CV/Nova             7945
Blazar              5085
SNIa                3956
SNIbc               1626
SNII                 890
SLSN                 727
Name: count, dtype: int64


#### Data sets
Nos interesa obtener grandes volumenes de datos para el entrenamiento, por lo tanto realizamos un codigo que se encarga de sacar variaos set de datos y juntarlos en un puro dataframe, vale recalcar que los datos son los extraidos anteriormente en binario (formato .npy)

In [ ]:
# Buscamos todos los archivos con datos completos
folders = [
    "/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps",
    "/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps_2",
    "/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps_3",
    "/home/seba/Usach/bigdata/Proyecto/ztf_3d_stamps_4"
]

rows = []

for folder in folders:
    file_list = glob.glob(os.path.join(folder, "alert_*_full.npy"))
    total_files = len(file_list)

    for j, path in enumerate(file_list, 1):
        try:
            data = np.load(path, allow_pickle=True).item()
            rows.append({
                "object_id": data["object_id"],
                "data": data["data"],
                **data["metadata"]  # Desempaquetamos el dict de metadatos como columnas
            })
            print(f'Archivos procesados: {j} de {total_files}  ', end='\r')
        except Exception as e:
            print(f"Error cargando {path}: {e}")


# Converción a DataFrame
avro_df = pd.DataFrame(rows)
avro_df = avro_df.drop_duplicates(subset='object_id', keep='first') # eliminamos duplicados con el mismo object_id

print(avro_df.shape)
print(avro_df.head())

# Cross Match
Ahora que tenemos el set de datos toca realizar el crossmatch, esto es importante ya que la red neuronal debe de saber que clase de evento transitorio está analizando (en nuestro caso nos decantamos por QSO, YSO , AGN, VS, SN y otros ), ademas de que el set de datos etiquetados no tiene imagenes y por lo tanto es completamente necesario realizar el crossmatch

In [ ]:
# Definimos el mapeo de clases a las nuevas etiquetas
label_map = {
    'AGN': 'AGN',
    'Blazar': 'AGN',
    'QSO': 'QSO',
    'YSO': 'YSO',
    'E': 'VS',
    'LPV': 'VS',
    'RRL': 'VS',
    'DSCT': 'VS',
    'CEP': 'VS',
    'SNIa': 'Other',
    'SNIbc': 'Other',
    'SNII': 'Other',
    'SLSN': 'Other',
    'Periodic-Other': 'Other',
    'CV/Nova': 'Other'
}

# Asignamos la nueva etiqueta según el mapeo, si no está en el mapeo se pone 'Other'
alerce_data['label'] = alerce_data['predicted_class'].map(label_map).fillna('Other')

# Seleccionamos solo las columnas necesarias y renombramos para el merge
alerce_labels = alerce_data[['oid', 'label']].rename(columns={'oid': 'object_id'})

# Cross-match con avro_data
final_avro_df = pd.merge(avro_df, alerce_labels, on='object_id', how='inner')

print(final_avro_df['label'].value_counts())
print(final_avro_df.head())

label
VS       97866
Other    24169
YSO       6931
QSO       2791
AGN       1931
SN         103
Name: count, dtype: int64
      object_id                                               data  \
0  ZTF19aamuloq  [[[269.03296, 272.72073, 276.7851, 271.75027, ...   
1  ZTF18abmrhkl  [[[98.008545, 98.33455, 103.00464, 93.19916, 8...   
2  ZTF19aaydqdp  [[[337.72638, 336.0949, 336.17847, 334.89038, ...   
3  ZTF18aakghjo  [[[143.20807, 146.21886, 154.38998, 142.27063,...   
4  ZTF18abbrojy  [[[374.30634, 382.51877, 375.64142, 376.90683,...   

           ra        dec     magpsf  sigmapsf  isdiffpos  diffmaglim  \
0  175.181255 -14.765624  19.262691  0.163901          1   19.746798   
1  328.547468  31.060491  19.858799  0.119543          0   20.517921   
2  209.058338 -26.614814  17.936068  0.090117          1   19.776381   
3  232.806765  70.447667  16.847271  0.050514          0   20.256754   
4  286.952025 -26.940841  16.846992  0.069402          0   19.772224   

       fwhm  sgscore1  .

#### Guardar Dataset en varios pickle
En nuestro caso nos acomoda guardar los datos formato pickle, ya que la estructura de nuestros datos es compleja debido a la incorporacion de metadatos con sus respectiva imagenes   

In [ ]:
# Filtramos por cada clase y tomamos una muestra eleatoria de 12000 ejemplos de la clase 'Other' y 'Variable_Star'para balancear el dataset

Other = final_avro_df[final_avro_df['label'] == 'Other'].sample(n=12000, random_state=42)
Other = Other.reset_index(drop=True)
Other['data'] = Other['data'].apply(normalize_data)

Quasar = final_avro_df[final_avro_df['label'] == 'QSO']
Quasar = Quasar.reset_index(drop=True)
Quasar['data'] = Quasar['data'].apply(normalize_data)

AGN = final_avro_df[final_avro_df['label'] == 'AGN']
AGN = AGN.reset_index(drop=True)
AGN['data'] = AGN['data'].apply(normalize_data)

YSO = final_avro_df[final_avro_df['label'] == 'YSO']
YSO = YSO.reset_index(drop=True)
YSO['data'] = YSO['data'].apply(normalize_data)

Variable_Star = final_avro_df[final_avro_df['label'] == 'VS'].sample(n=12000, random_state=42)
Variable_Star = Variable_Star.reset_index(drop=True)
Variable_Star['data'] = Variable_Star['data'].apply(normalize_data)


# Finalmente como nos indico nuestro especialista en redes neuronales, 
# guardaremos cada clase en un .pickle separado
Quasar.to_pickle("ztf_avro_Grande_QSO.pkl")
AGN.to_pickle("ztf_avro_Grande_AGN.pkl")
YSO.to_pickle("ztf_avro_Grande_YSO.pkl")
Other.to_pickle("ztf_avro_Grande_Other.pkl")
Variable_Star.to_pickle("ztf_avro_Grande_Variable_Star.pkl")

# En resumen
### el codigo lo que hace es:
- Procesar los datos de alerta ZTF desde archivos avro
- Extraer las imagenes en formato 3D (3 canales) para cada alerta (también se limpian los datos transformando los NaN a 0)
- Extraer una amplia selección de metadata de las imagenes ñas cuales servirán para el entrenamiento de la red neuronal (también nos aseguramos que no hayan datos nan reemplazando por 0 o por -999 según el tipo de dato)
- Unimos los datos con etiquetas entregadas por alerce (hacemos un crossmatch con un dataset pequeño el cual tiene datos etiquetados)
- Filtramos por clases de nuestro interes y agregamos una nueva clase para evitar error de clasificación (QSO, YSO , AGN, VS y otros)
- Exportamos el conjunto de datos etiquetados en pickle para futuros entrenamientos de la red neuronal